# K-Map Visualizer
This Jupyter notebook allows users to see how a kmap can be solved.

Simply input the matrix you want and press play.

In [307]:
from manim import *
import numpy as np
config.media_width = "100%"
_RV = "-v WARNING -qm --progress_bar None --disable_caching Example"

## Settings:

In [308]:
# Settings
input_matrix = \
   [[1, 1, 0, 1],
    [1, 0, 0, 0],]
   
   
predefined_colors = [PURPLE, MAROON, RED, ORANGE, GOLD, YELLOW, GREEN, TEAL, BLUE]

In [309]:
# K Map Solver
from enum import Enum
import random


class tileStatus(Enum):
  INVALID = 0
  UNEXPLORED = 1
  VISITED = 2
  

def create_super_k_matrix(matrix):
  matrix = np.array(matrix)
  # Get the dimensions of the matrix
  m, n = matrix.shape

  # Split indices
  mid_m, mid_n = m // 2, n // 2

  top_left, top_right       = matrix[:mid_m, :mid_n], matrix[:mid_m, mid_n:]
  bottom_left, bottom_right = matrix[mid_m:, :mid_n], matrix[mid_m:, mid_n:]
  
  top = np.hstack((bottom_right, bottom_left))
  bottom = np.hstack((top_right, top_left))
  recurssion_square = lambda x, y : x((y, y))
  
  super_matrix = recurssion_square(np.hstack, recurssion_square(np.vstack, np.vstack((top, bottom))))
  return super_matrix

def print_bool_array(array):
  print(np.matrix([[int(value) for value in row] for row in array]))
  
def color_selector(predefined_colors):
    selected_colors = set()
    
    def select_color():
        # If all predefined colors are selected, generate a new one
        if len(selected_colors) == len(predefined_colors):
            new_color = "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            return new_color
        else:
            # Select a unique color from the predefined list
            color = random.choice([color for color in predefined_colors if (color.to_hex() if type(color) is ManimColor else str(color)) not in selected_colors])
            selected_colors.add(color.to_hex() if type(color) is ManimColor else str(color))
            return color
    
    return select_color

def create_min_groups(matrix: list[list[int]]):
  matrix = np.array(matrix)
  
  super_matrix = create_super_k_matrix(matrix)
  m, n = super_matrix.shape
  visited = [[False for _ in range(n)] for _ in range(m)]
  
  min_terms = []
  coord_shift = lambda x, k, shift: x % (k//2) + shift//2
  def is_valid(x, y):

    return super_matrix[y][x] == 1 and 0 <= x < m and 0 <= y < n 

  def is_filled(matrix, top_left, bottom_right, signature = 1):
    start_row, start_col = top_left
    end_row, end_col = bottom_right

    for x in range(start_col, end_col + 1):
      for y in range(start_row, end_row + 1):
        if matrix[x][y] != signature:
            return False
    return True


  def find_rectangle(start_x, start_y):

    max_x, max_y = start_x, start_y
    extension_x, extension_y = 0, 0
    find_max_y, find_max_x = tileStatus.UNEXPLORED, tileStatus.UNEXPLORED
    use_remaining = False
    
    def satisfy_y(max_x, max_y, extension_x, extension_y):

      max_y = start_y + 1 if extension_y == 0 else extension_y * 2
      if max_y >= m:
        return tileStatus.INVALID
      
      y_unexplored = not is_filled(visited, (start_x, start_y), (max_x, max_y), True)

      y_filled     = is_filled(super_matrix, (start_x, start_y), (max_x, max_y))
      

      if y_unexplored and y_filled and (extension_y <= m/2):
        return tileStatus.UNEXPLORED
      elif y_filled and (extension_y <= m/2):
        return tileStatus.VISITED
      else:
        return tileStatus.INVALID
      
    def satisfy_x(max_x, max_y, extension_x, extension_y):

      max_x = start_x + 1 if extension_x == 0 else extension_x * 2
      if max_x>= n :
        return tileStatus.INVALID
      
      x_unexplored  = not is_filled(visited, (start_x, start_y), (max_x, max_y), True)
      x_filled      = is_filled(super_matrix, (start_x, start_y), (max_x, max_y))
        
      if (x_unexplored and x_filled and (extension_x <= n//2)):
        return tileStatus.UNEXPLORED
      elif x_filled and (extension_x<= n//2):
        return tileStatus.VISITED
      else:
        return tileStatus.INVALID
    
    while find_max_y != tileStatus.INVALID or find_max_x != tileStatus.INVALID:
      possible_expansion = lambda tile: tile == tileStatus.UNEXPLORED or tile == tileStatus.VISITED and use_remaining
      find_max_x = satisfy_x(max_x, max_y, extension_x, extension_y)
      if possible_expansion(find_max_x):
        if extension_x == 0:
          extension_x = 1
        else:
          extension_x *= 2
        
        max_x = start_x + extension_x // 2
      
      find_max_y = satisfy_y(max_x, max_y, extension_x, extension_y)
      if possible_expansion(find_max_y):
        if extension_y == 0:
          extension_y = 1
        else:
          extension_y *= 2
        max_y = start_y + extension_y // 2 

      
      if find_max_y != tileStatus.UNEXPLORED and find_max_x != tileStatus.UNEXPLORED:
        use_remaining = True
      
    return (start_x, start_y, max_x, max_y)

  for y in range(m):
    for x in range(n):
      if not is_valid(x, y):
        continue
      
      new_rect = find_rectangle(x, y)
      (start_x, start_y, max_x, max_y) = new_rect
      
      submatrix = [row[new_rect[0]:new_rect[2] + 1] for row in visited[new_rect[1]:new_rect[3] + 1]]
      
      if not any(not all(row) for row in submatrix):
        continue
      groups = 0
      # Mark cells as visited
      for ky in range(start_y, max_y + 1):
        for kx in range(start_x, max_x + 1):
            xs = lambda shift: coord_shift(kx, n, shift) # xshifted
            ys = lambda shift: coord_shift(ky, m, shift) # yshifted
            visited[ys(0)][xs(n)] = True
            visited[ys(m)][xs(0)] = True
            visited[ys(m)][xs(n)] = True
            visited[ys(0)][xs(0)] = True

      coords = lambda shift_x, shift_y: [(start_x - n// 4 + shift_x, start_y - m//4 + shift_y), (max_x - n// 4 + shift_x, max_y - m//4 + shift_y)]
      min_terms.append([coords(0, 0), coords(n//2, 0), coords(0, m//2), coords(n//2, m//2)])


  return min_terms


In [310]:
# Adds colors from the solved groups      
mingroups = create_min_groups(input_matrix)
mop_mingroup = []
color_picker = color_selector(predefined_colors)
for mingroup_cluster in mingroups:
  selected_color = color_picker()
  for mingroup in mingroup_cluster:
    mop_mingroup.append({"coords": mingroup, "color": selected_color})

## Animation Output:

In [311]:
%%manim $_RV
# Animation manim magic to see
class Example(Scene):
    def construct(self):
        # Example matrix (you can replace this with any n x m matrix)
        matrix = input_matrix

        # Call the function to generate the K-map
        kmap, labels = self.create_kmap(matrix)

        # Add a title
        title = Text("Karnaugh Map", font_size=24).to_edge(UP)
        self.play(Write(title))

        # Add the K-map labels and grid to the scene
        self.play(Create(labels), Create(kmap))


        self.wait(1)
        
        # Define rectangles using top-left and bottom-right coordinates
        rectangles = mop_mingroup

        # Get the top-left corner of the K-map
        kmap_corner = kmap.get_corner(UL)  # UL = Upper Left in Manim
        kmap_bottom_right = kmap.get_corner(DR)  # DR = Down Right in Manim

        # Constants for cell dimensions
        cell_width = 1.0
        cell_height = 1.0
        cell_margin = 0
        corner_radius = 0.5
        dashes = 10


        # Add encompassing rectangle around the entire K-map
        kmap_width = kmap_bottom_right[0] - kmap_corner[0]
        kmap_height = kmap_corner[1] - kmap_bottom_right[1]
        kmap_rect = Rectangle(
            width=kmap_width,
            height=kmap_height,
            color=WHITE,
            fill_opacity=0.0,  # No fill, only border
             stroke_opacity=0.1,
        )
        kmap_rect.move_to(kmap.get_center())
        
        dashed_kmap_rect = DashedVMobject(kmap_rect, num_dashes=40)


        
        self.play(Create(kmap_rect))

        # Draw the rectangles with alpha transparency, clipped within the K-map boundaries
        for i, rect in enumerate(rectangles):
            # Unpack the given coordinates
            (coord1, coord2) = rect["coords"]
            color = rect["color"]

            # Calculate the most encompassing rectangle's corners
            top_left_row = min(coord1[1], coord2[1])
            top_left_col = min(coord1[0], coord2[0])
            bottom_right_row = max(coord1[1], coord2[1])
            bottom_right_col = max(coord1[0], coord2[0])

            # Calculate rectangle coordinates
            top_left_x = kmap_corner[0] + top_left_col * (cell_width)
            top_left_y = kmap_corner[1] - top_left_row * (cell_height)
            bottom_right_x = kmap_corner[0] + (bottom_right_col + 1) * (cell_width)
            bottom_right_y = kmap_corner[1] - (bottom_right_row + 1) * (cell_height)


            # Recalculate width, height, and center after clipping
            width = bottom_right_x - top_left_x + cell_margin 
            height = top_left_y - bottom_right_y + cell_margin 
            center_x = (top_left_x + bottom_right_x) / 2
            center_y = (top_left_y + bottom_right_y) / 2

            # Create the filled rectangle with transparency (alpha=0.2)
            rectangle = RoundedRectangle(
                corner_radius=min(corner_radius, width / 2, height / 2),
                width=width,
                height=height,
                color=color,
                stroke_opacity=0.3,
                fill_opacity=0.3  # Set transparency
            )
            
            rectangle.move_to([center_x, center_y, 0])
            clipped_rectangle = Intersection(rectangle, kmap_rect, color=color, fill_opacity = 0.1, stroke_opacity=0.3)
            # Add the rectangle to the scene
            self.play(Create(clipped_rectangle), run_time=0.5)
            

        self.wait(3)

    def create_kmap(self, matrix):
        """
        Create a flat Karnaugh Map representation for an n x m matrix with Gray code labels.
        """
        n = len(matrix)      # Number of rows
        m = len(matrix[0])   # Number of columns

        # Constants for layout
        cell_width = 1.0
        cell_height = 1.0
        cell_margin = 0

        # Group to hold all K-map elements
        kmap_group = VGroup()
        labels_group = VGroup()

        # Gray code generation for labels
        def gray_code(num_bits):
            return [bin(i ^ (i >> 1))[2:].zfill(num_bits) for i in range(2 ** num_bits)]

        row_labels = gray_code(int.bit_length(max(1, n - 1)))
        col_labels = gray_code(int.bit_length(max(1, m - 1)))

        get_color = lambda x: YELLOW if x == 1 else GRAY
        get_bold  = lambda x: NORMAL if x == 1 else NORMAL
        
        # Generate the grid of the K-map
        for i, row in enumerate(matrix):
            for j, value in enumerate(row):
                # Create the rectangle for the cell
                rect = Rectangle(width=cell_width, height=cell_height, color=GRAY,  stroke_opacity=0.1)
                rect.move_to(np.array([(j - m / 2) * (cell_width + cell_margin) + cell_width / 2,
                                       -(i - n / 2) * (cell_height + cell_margin) - cell_height / 2,
                                       0]))

                # Add the value as a text in the center of the cell
          
                text = Text(str(value), font_size=36, color=get_color(value), weight=get_bold(value))
                text.set_z_index(1)
                text.move_to(rect.get_center())

                # Group the rectangle and text together
                cell_group = VGroup(rect, text)

                # Add to the overall K-map group
                kmap_group.add(cell_group)

        def add_table_labels(idx, label, labelType: str):
            label_text = Text(label, font_size=24)
            is_col = labelType == "COL"
            label_text.next_to(kmap_group[0].get_top() if is_col else kmap_group[0].get_left() , 
                               UP if is_col else LEFT, 
                               buff=0.5)
            label_text.shift((RIGHT if is_col else DOWN) * idx * 
                             (cell_width if is_col else cell_height + cell_margin))
            labels_group.add(label_text)
        
        # Add row labels
        for i, label in enumerate(row_labels):
            add_table_labels(i, label, "ROW")

        # Add column labels
        for j, label in enumerate(col_labels):
            add_table_labels(j, label, "COL")

        return kmap_group, labels_group

Manim Community v0.18.1